In [38]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [39]:
time_intervals = [(x, x+3) for x in range(0, 24) if x%3 == 0]
print(time_intervals)

[(0, 3), (3, 6), (6, 9), (9, 12), (12, 15), (15, 18), (18, 21), (21, 24)]


In [40]:
weekday_intervals = list(range(1, 8))
print(weekday_intervals)

[1, 2, 3, 4, 5, 6, 7]


In [41]:
space_intervals = [{'x': (0, 5), 'y': (0, 2)}, 
                   {'x': (0, 5), 'y': (2, 4)}, 
                   {'x': (0, 5), 'y': (4, 6)}, 
                   {'x': (0, 5), 'y': (6, 8)}, 
                   {'x': (0, 5), 'y': (8, 11)}, 
                   {'x': (5, 11), 'y': (0, 2)},
                   {'x': (5, 11), 'y': (2, 4)},
                   {'x': (5, 11), 'y': (4, 6)},
                   {'x': (5, 11), 'y': (6, 8)},
                   {'x': (5, 11), 'y': (8, 11)}]

print(space_intervals)

[{'y': (0, 2), 'x': (0, 5)}, {'y': (2, 4), 'x': (0, 5)}, {'y': (4, 6), 'x': (0, 5)}, {'y': (6, 8), 'x': (0, 5)}, {'y': (8, 11), 'x': (0, 5)}, {'y': (0, 2), 'x': (5, 11)}, {'y': (2, 4), 'x': (5, 11)}, {'y': (4, 6), 'x': (5, 11)}, {'y': (6, 8), 'x': (5, 11)}, {'y': (8, 11), 'x': (5, 11)}]


In [42]:
def get_all_combinations(time_ints, weekday_ints, space_ints):
    ans_dicts = []
    for t in time_ints:
        for w in weekday_ints:
            for s in space_ints:
                ans_dicts.append({'time_ints': t, 'weekday_ints': w, 'space_ints': s})
    return ans_dicts

In [43]:
features_grid = get_all_combinations(time_intervals, weekday_intervals, space_intervals)

In [44]:
features_grid[0]

{'space_ints': {'x': (0, 5), 'y': (0, 2)},
 'time_ints': (0, 3),
 'weekday_ints': 1}

In [71]:
len(time_intervals)

8

In [58]:
d = df_test.hour.apply(int).value_counts()

In [65]:
d = pd.DataFrame(d)

In [67]:
d.sort_index()

,hour
1,358564
2,360219
3,359809
4,360061
5,360557
6,359248
7,358103
8,358026
9,358515
10,359316


In [45]:
def fgeneration_data(df):
    mintue = df['time']%60
    df['hour'] = df['time']//60
    df['weekday'] = df['hour']//24
    df['month'] = df['weekday']//30
    df['year'] = df['weekday']//365+1
    df['hour'] = ((df['hour']%24+1)+mintue/60.0)
    df['weekday'] = df['weekday']%7+1
    df['month'] = df['month']%12+1
    df.drop(['time'], axis=1, inplace=True)
    dfo = df
    return dfo

def preprocessing_data(df, fw, std_val):
    df['x'] = df['x']*fw['x']/std_val['x']
    df['y'] = df['y']*fw['y']/std_val['y']
    df['hour'] = df['hour']*fw['hour']/std_val['hour']
    df['weekday'] = df['weekday']*fw['weekday']/std_val['weekday']
    df['year'] = df['year']*fw['year'] #dont needed
    df['month'] = df['month']*fw['month']/std_val['month']
    dfo = df
    return dfo

# Classification inside one grid cell.
def process_one_cell(df_train, df_test, grid_condition, fw, std_val):
    # {'accuracy_ints': (0, 18), 'time_ints': (0, 3), 'weekday_ints': 4} = grid_condition
#     accuracy_min, accuracy_max = grid_condition['accuracy_ints']
    x_min, x_max = grid_condition['space_ints']['x']
    y_min, y_max = grid_condition['space_ints']['y']
    hh_min, hh_max = grid_condition['time_ints']
    weekday_v = int(grid_condition['weekday_ints'])
    #Working on df_train
    filter_df = lambda df: df[(df['hour'] >= hh_min) & (df['hour'] < hh_max) &
                              (df['x'] >= x_min) & (df['x'] < x_max) &
                              (df['y'] >= y_min) & (df['y'] < y_max) &
                                (df['weekday'] == weekday_v)]
    df_cell_train = filter_df(df_train)
    df_cell_train = preprocessing_data(df_cell_train, fw, std_val)
    
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    #Working on df_test
    # to be delete: df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    df_cell_test = filter_df(df_test)
    df_cell_test = preprocessing_data(df_cell_test, fw, std_val)
    row_ids = df_cell_test.index
 
    #Preparing data
    le = LabelEncoder()
    y = le.fit_transform(df_cell_train.place_id.values)
    X = df_cell_train.drop(['place_id'], axis=1).values.astype(float)
    X_test = df_cell_test.values.astype(float)

    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=26, weights='distance', 
                               metric='manhattan')
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3]) 
    
    return pred_labels, row_ids
   

def process_grid(df_train, df_test, features_grid, fw, std_val):
    """
    Iterates over all grid cells, aggregates the results and makes the
    submission.
    """ 
    preds = np.zeros((df_test.shape[0], 3), dtype=np.int64)
    l = len(features_grid)
   
    for i, it in enumerate(features_grid):
        #Applying classifier to one grid cell
        #TODO: add preparation
        
        pred_labels, row_ids = process_one_cell(df_train, df_test, it, fw, std_val)
        #Updating predictions
        preds[row_ids] = pred_labels
        print("STEP={0} FROM={1}".format(*[i, l]))
        #print("Row %d/%d elapsed time: %.2f seconds" % (i+1, (int)(size/x_step),(time.time() - start_time_row)))

    print('Generating submission file ...')
    #Auxiliary dataframe with the 3 best predictions for each sample
    df_aux = pd.DataFrame(preds, dtype=str, columns=['l1', 'l2', 'l3'])  
    
    #Concatenating the 3 predictions for each sample
    ds_sub = df_aux.l1.str.cat([df_aux.l2, df_aux.l3], sep=' ')
    
    #Writting to csv
    ds_sub.name = 'place_id'
    ds_sub.to_csv('/opt/devs/sub_knn_grid_alex4.csv', index=True, header=True, index_label='row_id')  


In [ ]:
!git add -A; git commit -

In [46]:
print('Loading data ...')

df_train = fgeneration_data(pd.read_csv('/opt/devs/train_facebook.csv',
                       usecols=['row_id','x','y', 'time','place_id'],
                       index_col = 0))

train_sd = df_train.std()
fw = {'x': 3, 'y': 7, 'hour': 2, 'weekday': 1, 'month': 0.5, 'year': 1} #feature weights (black magic here)
th = 5 #Keeping place_ids with more than th samples.

df_test = fgeneration_data(pd.read_csv('/opt/devs/test_facebook.csv',
                      usecols=['row_id','x','y', 'time'],
                      index_col = 0))

Loading data ...


In [47]:

print('Preparing train data')
#Feature engineering

#Solving classification problems inside each grid cell 
process_grid(df_train, df_test, features_grid, fw, train_sd)

Preparing train data
STEP=0 FROM=560
STEP=1 FROM=560
STEP=2 FROM=560
STEP=3 FROM=560
STEP=4 FROM=560
STEP=5 FROM=560
STEP=6 FROM=560
STEP=7 FROM=560
STEP=8 FROM=560
STEP=9 FROM=560
STEP=10 FROM=560
STEP=11 FROM=560
STEP=12 FROM=560
STEP=13 FROM=560
STEP=14 FROM=560
STEP=15 FROM=560
STEP=16 FROM=560
STEP=17 FROM=560
STEP=18 FROM=560
STEP=19 FROM=560
STEP=20 FROM=560
STEP=21 FROM=560
STEP=22 FROM=560
STEP=23 FROM=560
STEP=24 FROM=560
STEP=25 FROM=560
STEP=26 FROM=560
STEP=27 FROM=560
STEP=28 FROM=560
STEP=29 FROM=560
STEP=30 FROM=560
STEP=31 FROM=560
STEP=32 FROM=560
STEP=33 FROM=560
STEP=34 FROM=560
STEP=35 FROM=560
STEP=36 FROM=560
STEP=37 FROM=560
STEP=38 FROM=560
STEP=39 FROM=560
STEP=40 FROM=560
STEP=41 FROM=560
STEP=42 FROM=560
STEP=43 FROM=560
STEP=44 FROM=560
STEP=45 FROM=560
STEP=46 FROM=560
STEP=47 FROM=560
STEP=48 FROM=560
STEP=49 FROM=560
STEP=50 FROM=560
STEP=51 FROM=560
STEP=52 FROM=560
STEP=53 FROM=560
STEP=54 FROM=560
STEP=55 FROM=560
STEP=56 FROM=560
STEP=57 FROM=560
STE